In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import gc
import math
import copy
import random
import itertools
import argparse
from tqdm import tqdm



import numpy as np
import pandas as pd
from math import floor
from functools import reduce
from scipy.stats import ttest_ind

import sklearn.metrics as metrics
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    brier_score_loss,
    roc_auc_score,
    average_precision_score,
    precision_recall_curve
)
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

import torch
from torch import nn
from torch import linalg as LA
import torch.nn.functional as F
from torch.nn import init
from torch.autograd import Variable
import torch.utils.data
from torch.utils.data.dataset import Dataset
from torch.utils.data.sampler import SubsetRandomSampler, SequentialSampler

import torchvision.transforms as transforms

from tqdm import tqdm

# Set up device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [2]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.best_model = None

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = model
            self.val_loss_min = val_loss
        elif score < self.best_score - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.val_loss_min = val_loss
            self.best_model = model
            self.counter = 0

In [3]:
class PCGrad():
    def __init__(self, optimizer, reduction='mean'):
        self._optim, self._reduction = optimizer, reduction
        return

    @property
    def optimizer(self):
        return self._optim

    def zero_grad(self):
        return self._optim.zero_grad(set_to_none=True)

    def step(self):
        return self._optim.step()

    def pc_backward(self, objectives):
        grads, shapes, has_grads = self._pack_grad(objectives)
        pc_grad = self._project_conflicting(grads, has_grads)
        pc_grad = self._unflatten_grad(pc_grad, shapes[0])
        self._set_grad(pc_grad)
        return

    def _project_conflicting(self, grads, has_grads, shapes=None):
        shared = torch.stack(has_grads).prod(0).bool()
        pc_grad, num_task = copy.deepcopy(grads), len(grads)
        for g_i in pc_grad:
            random.shuffle(grads)
            for g_j in grads:
                g_i_g_j = torch.dot(g_i, g_j)
                if g_i_g_j < 0:
                    g_i -= (g_i_g_j) * g_j / (g_j.norm() ** 2)
        merged_grad = torch.zeros_like(grads[0]).to(grads[0].device)
        if self._reduction:
            merged_grad[shared] = torch.stack([g[shared]
                                               for g in pc_grad]).mean(dim=0)
        elif self._reduction == 'sum':
            merged_grad[shared] = torch.stack([g[shared]
                                               for g in pc_grad]).sum(dim=0)
        else:
            exit('invalid reduction method')

        merged_grad[~shared] = torch.stack([g[~shared]
                                            for g in pc_grad]).sum(dim=0)
        return merged_grad

    def _set_grad(self, grads):
        idx = 0
        for group in self._optim.param_groups:
            for p in group['params']:
                p.grad = grads[idx]
                idx += 1
        return

    def _pack_grad(self, objectives):
        grads, shapes, has_grads = [], [], []
        for obj in objectives:
            self._optim.zero_grad(set_to_none=True)
            obj.backward(retain_graph=True)
            grad, shape, has_grad = self._retrieve_grad()
            grads.append(self._flatten_grad(grad, shape))
            has_grads.append(self._flatten_grad(has_grad, shape))
            shapes.append(shape)
        return grads, shapes, has_grads

    def _unflatten_grad(self, grads, shapes):
        unflatten_grad, idx = [], 0
        for shape in shapes:
            length = np.prod(shape)
            unflatten_grad.append(grads[idx:idx + length].view(shape).clone())
            idx += length
        return unflatten_grad

    def _flatten_grad(self, grads, shapes):
        flatten_grad = torch.cat([g.flatten() for g in grads])
        return flatten_grad

    def _retrieve_grad(self):
        grad, shape, has_grad = [], [], []
        for group in self._optim.param_groups:
            for p in group['params']:
                # if p.grad is None: continue
                # tackle the multi-head scenario
                if p.grad is None:
                    shape.append(p.shape)
                    grad.append(torch.zeros_like(p).to(p.device))
                    has_grad.append(torch.zeros_like(p).to(p.device))
                    continue
                shape.append(p.grad.shape)
                grad.append(p.grad.clone())
                has_grad.append(torch.ones_like(p).to(p.device))
        return grad, shape, has_grad

In [4]:
def seed_everything(seed):
    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)  
    random.seed(seed) 
seed_everything(111)

In [5]:
class CNN_1(nn.Module):
    def __init__(self, dim, int_dim, shell):
        super(CNN_1, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=shell, stride=shell, padding=0),
            nn.BatchNorm1d(16)
        )
        self.fc = nn.Linear(dim, int_dim)
        self.fc2 = nn.Linear(int_dim, 1)

        self.act = nn.Tanh()
        self.pool = nn.MaxPool1d(2, stride=2)

    def forward(self, x):

        x = x.view(x.shape[0], 1, -1)
        out = self.conv(x)
        out = self.act(out)
        out = self.pool(out)
        out = out.view(x.shape[0], out.size(1) * out.size(2))
        feat = self.fc(out)
        logit = self.fc2(feat)
        logit = logit.view(-1, )

        return logit, feat


class CNN_2(nn.Module):
    def __init__(self, dim, int_dim, shell):
        super(CNN_2, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=shell, stride=shell, padding=0),
            nn.BatchNorm1d(16)
        )
        self.fc = nn.Linear(dim, int_dim)
        self.fc2 = nn.Linear(int_dim, 1)

        self.act = nn.Tanh()
        self.pool = nn.MaxPool1d(2, stride=2)

    def forward(self, x):
        x = x.view(x.shape[0], 1, -1)
        out = self.conv(x)
        out = self.act(out)
        out = self.pool(out)
        out = out.view(x.shape[0], out.size(1) * out.size(2))
        feat = self.fc(out)
        logit = self.fc2(feat)
        logit = logit.view(-1, )

        return logit, feat

In [6]:
def train_model(train_x, valid_x, tr_y, val_y, model, patience, n_epochs, optimizer, rg):
    train_losses = []
    valid_losses = []
    avg_train_losses = []
    avg_valid_losses = []

    early_stopping = EarlyStopping(patience=patience, verbose=True)
    model.train()

    for epoch in range(1, n_epochs + 1):

        ###################
        # train the model #
        ###################
        optimizer.zero_grad()

        output, feat = model(train_x)
        # calculate the loss
        alpha_t = 0.8 * ((early_stopping.counter + 1) / patience)
        alpha_t = max(0, alpha_t)
        if epoch == 1:
            before_predict = tr_y.float()

        loss1 = loss_func(output, tr_y.float())
        target = Variable(torch.where(tr_y.float() > 0, 1, -1))
        loss2 = loss_func2(torch.sigmoid(output), before_predict, target)
        loss = ((1 - alpha_t) * loss1 + (alpha_t) * loss2)

        norm = torch.cuda.FloatTensor([0])
        for parameter in model.parameters():
            norm += torch.norm(parameter, p=1)

        loss = loss + norm * rg

        before_predict = torch.sigmoid(output).detach()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

        ######################
        # validate the model #
        ######################
        with torch.no_grad():
            model.eval()
            output, feat = model(valid_x)
            loss = loss_func(output, val_y.float())

            # record validation loss
            valid_losses.append(loss.item())

        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        epoch_len = len(str(n_epochs))

        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        #print(print_msg)

        train_losses = []
        valid_losses = []
        early_stopping(valid_loss, model)

        if early_stopping.early_stop:
            break

    return early_stopping.best_model, early_stopping.val_loss_min

In [7]:
class GCN(nn.Module):
    def __init__(self, num_feature, num_node, bias=False):
        super(GCN, self).__init__()
        self.conv1 = nn.Conv1d(num_node, num_node, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(num_feature, num_feature, kernel_size=1, bias=bias)

    def forward(self, x):
        h = self.conv1(x)
        h = h - x
        h = self.relu(self.conv2(h.permute(0, 2, 1)).permute(0, 2, 1))
        return h

class BilateralGCN(nn.Module):
    def __init__(self, in_feature, num_node):
        super().__init__()
        self.gcn = GCN(in_feature, num_node)

    def forward(self, x, y):
        fusion = x + y
        fusion = self.gcn(fusion)
        x = x + fusion
        y = y + fusion

        return x, y

class clffusion(nn.Module):
    def __init__(self, in_dim):
        super(clffusion, self).__init__()
        self.num_n = in_dim // 4
        self.num_s = in_dim // 2

        self.conv1 = nn.Conv1d(in_dim, self.num_n, kernel_size=1)
        self.conv2 = nn.Conv1d(in_dim, self.num_s, kernel_size=1)

        self.BGCN = BilateralGCN(self.num_s, self.num_n)

        self.conv = nn.Conv1d(self.num_s, in_dim, kernel_size=1)
        self.bn = nn.BatchNorm1d(in_dim)
        self.in_dim = in_dim

        self.clf1 = nn.Linear(in_dim, 1)
        self.clf2 = nn.Linear(in_dim, 1)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x1, x2):
        n = x1.size(0)
        e = torch.bmm(x1.reshape(n, 1, -1).permute(0, 2, 1), x2.reshape(n, 1, -1))
        s1 = self.softmax(e.permute(0, 2, 1)).permute(0, 2, 1)
        s2 = self.softmax(e).permute(0, 2, 1)

        x1 = torch.bmm(x1.reshape(n, 1, -1), s1)
        x2 = torch.bmm(x2.reshape(n, 1, -1), s2)

        x1 = x1.permute(0, 2, 1)
        x2 = x2.permute(0, 2, 1)

        x1_v = self.conv1(x1)
        x2_v = self.conv1(x2)
        x1_w = self.conv2(x1)
        x2_w = self.conv2(x2)

        x1_g = torch.bmm(x1_v, torch.transpose(x1_w, 1, 2))
        x2_g = torch.bmm(x2_v, torch.transpose(x2_w, 1, 2))

        x1_g, x2_g = self.BGCN(x1_g, x2_g)
        x1_g = torch.bmm(torch.transpose(x1_v, 1, 2), x1_g).view(n, self.num_s, -1)
        x2_g = torch.bmm(torch.transpose(x2_v, 1, 2), x2_g).view(n, self.num_s, -1)

        x1_fusion = self.conv(x1_g)
        x2_fusion = self.conv(x2_g)

        x1_fusion = F.relu(self.bn(x1_fusion))
        x2_fusion = F.relu(self.bn(x2_fusion))

        res1 = (x1 + x1_fusion).view(n, self.in_dim)
        res2 = (x2 + x2_fusion).view(n, self.in_dim)

        logit1 = self.clf1(res1).view(-1)
        logit2 = self.clf2(res2).view(-1)

        return logit1, logit2

In [8]:
def train_fusion_model(tr1, tr2, val1, val2, clf, patience, n_epochs, tr_y_clf, val_y, hard_ind, opt):
    train_losses = []
    valid_losses = []

    early_stopping = EarlyStopping(patience=patience, verbose=True)
    clf.train()
    for epoch in range(1, n_epochs + 1):

        if early_stopping.counter < (patience//2):
            tr_x1 = tr1[~hard_ind]
            tr_x2 = tr2[~hard_ind]
            tr_y = tr_y_clf[~hard_ind]
        else:
            tr_x1 = tr1
            tr_x2 = tr2
            tr_y = tr_y_clf

        ###################
        # train the model #
        ###################
        opt.zero_grad()
        output1, output2 = clf(tr_x1, tr_x2)

        loss1 = loss_func(output1, tr_y.float())
        loss2 = loss_func(output2, tr_y.float())
        loss = loss1 + loss2
        losses = [loss1, loss2]

        opt.zero_grad()
        opt.pc_backward(losses)
        opt.step()
        train_losses.append(loss.item())

        ######################
        # validate the model #
        ######################
        with torch.no_grad():
            clf.eval()
            output1, output2 = clf(val1, val2)

            loss1 = loss_func(output1, val_y.float())
            loss2 = loss_func(output2, val_y.float())
            loss = loss1 + loss2

            valid_losses.append(loss.item())
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)

        train_losses = []
        valid_losses = []

        early_stopping(valid_loss, clf)

        if early_stopping.early_stop:
            break
    best_clf = early_stopping.best_model

    return best_clf, early_stopping.val_loss_min

In [9]:
def weights_init(m):
    if isinstance(m, nn.Conv1d):
        torch.nn.init.xavier_uniform_(m.weight)
        # torch.nn.init.zeros_(m.bias)
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        # torch.nn.init.zeros_(m.bias)


In [10]:
rg = 0.001
learning_rate = 0.0001
lr1 = 0.0001
lr2 = 0.0001
n_epochs = 3000
patience = 30

type = './data/'
x1 = pd.read_csv(type + 'mRNA.csv', index_col=0, delimiter=',')
x2 = pd.read_csv(type + 'miRNA.csv', index_col=0, delimiter=',')
label = pd.read_csv(type + 'label.csv',  index_col=0,delimiter=',')

print(x1.shape)
print(x2.shape)
x1 = x1.fillna(0)
x2 = x2.fillna(0)
labels = np.array(label).flatten().astype(np.float32)

shell_list = list(range(10, 300, 30))
int_dim_list = [300, 100, 50]
rg_list = [0.0, 0.001,0.0001]

loss_func = nn.BCEWithLogitsLoss(weight=None, size_average=None, reduce=None, reduction='mean')
loss_func2 = nn.MarginRankingLoss()

(17996, 3078)
(743, 3078)


In [11]:
n_f1 = x1.shape[0]
n_f2 = x2.shape[0]

x1 = torch.tensor(x1.values)
x2 = torch.tensor(x2.values)

t1_res = []
t2_res = []

for it in range(0, 10):

    t1_auc = []
    t2_auc = []

    kf = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=it)
    train_index = []
    valid_index = []
    test_index = []
    
    for tr_index, te_index in kf.split(np.transpose(x1), labels):
        random.shuffle(tr_index)
        tmp = int(0.2 * len(tr_index))

        train_index.append(tr_index[tmp + 1:])
        valid_index.append(tr_index[0:tmp])
        test_index.append(te_index)

    seed_everything(it)

    for fold in range(1, 6):
        print("------------- Fold : " + str(fold) + "-------------")
        y = torch.tensor(labels)
        train_y = y[train_index[fold - 1]]
        valid_y = y[valid_index[fold - 1]]
        test_y = y[test_index[fold - 1]]

        train_y = Variable(train_y.type(torch.LongTensor)).to(device)
        valid_y = Variable(valid_y.type(torch.LongTensor)).to(device)
        test_y = Variable(test_y.type(torch.LongTensor))

        tr_x1 = x1[:, train_index[fold - 1]].t()
        val_x1 = x1[:, valid_index[fold - 1]].t()
        te_x1 = x1[:, test_index[fold - 1]].t()
        tr_x1 = torch.reshape(tr_x1, (tr_x1.shape[0], tr_x1.shape[1], 1))
        val_x1 = torch.reshape(val_x1, (val_x1.shape[0], val_x1.shape[1], 1))
        te_x1 = torch.reshape(te_x1, (te_x1.shape[0], te_x1.shape[1], 1))

        tr_x1 = Variable(tr_x1.type(torch.FloatTensor)).to(device)
        val_x1 = Variable(val_x1.type(torch.FloatTensor)).to(device)
        te_x1 = Variable(te_x1.type(torch.FloatTensor)).to(device)

        tr_x2 = x2[:, train_index[fold - 1]].t()
        val_x2 = x2[:, valid_index[fold - 1]].t()
        te_x2 = x2[:, test_index[fold - 1]].t()

        tr_x2 = torch.reshape(tr_x2, (tr_x2.shape[0], tr_x2.shape[1], 1))
        val_x2 = torch.reshape(val_x2, (val_x2.shape[0], val_x2.shape[1], 1))
        te_x2 = torch.reshape(te_x2, (te_x2.shape[0], te_x2.shape[1], 1))

        tr_x2 = Variable(tr_x2.type(torch.FloatTensor)).to(device)
        val_x2 = Variable(val_x2.type(torch.FloatTensor)).to(device)
        te_x2 = Variable(te_x2.type(torch.FloatTensor)).to(device)

        best_val = 9999
        for in_dim in tqdm(range(0, len(int_dim_list))):
            int_dim = int_dim_list[in_dim]

            # Omics1
            best_val1 = 9999
            for s in range(0, len(shell_list)):
                for r in range(0, len(rg_list)):
                    rg = rg_list[r]
                    shell = shell_list[s]
                    dim = 16 * int(int(n_f1 / shell) / 2)
                    model = CNN_1(dim, int_dim, shell).to(device)
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr1)
                    m1, valid_loss = train_model(tr_x1, val_x1, train_y, valid_y, model, patience, 1000,
                                                           optimizer, rg)
                    if valid_loss < best_val1:
                        best_val1 = valid_loss
                        best_model1 = m1
                        shell1 = shell
            # Embedding
            logit_x1, tr_f1 = best_model1(tr_x1)
            _, val_f1 = best_model1(val_x1)
            _, te_f1 = best_model1(te_x1)

            # Omics2
            best_val2 = 9999
            for s in range(0, len(shell_list)):
                for r in range(0, len(rg_list)):
                    rg = rg_list[r]
                    shell = shell_list[s]
                    dim = 16 * int(int(n_f2 / shell) / 2)
                    model = CNN_2(dim, int_dim, shell).to(device)
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr2)

                    m2, valid_loss = train_model(tr_x2, val_x2, train_y, valid_y, model, patience, 1000,
                                                           optimizer, rg)
                    if valid_loss < best_val2:
                        best_val2 = valid_loss
                        best_model2 = m2
                        shell2 = shell
            # Embedding
            logit_x2, tr_f2 = best_model2(tr_x2)
            _, val_f2 = best_model2(val_x2)
            _, te_f2 = best_model2(te_x2)

            # Delete for memory
            del (m1)
            del (m2)
            del (best_model1)
            del (best_model2)
            gc.collect()

            res1 = abs(train_y - torch.sigmoid(logit_x1))
            logit_mean1 = torch.mean(res1)
            ind1 = (res1 > logit_mean1)
            res2 = abs(train_y - torch.sigmoid(logit_x2))
            logit_mean2 = torch.mean(res2)
            ind2 = (res2 > logit_mean2)

            # Training fusion block
            inter_ind = (ind1 & ind2)
            clf = clffusion(int_dim).cuda()
            optimizer_f = PCGrad(torch.optim.SGD(clf.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005,
                                                 nesterov=True))
            best, f_val_loss = train_fusion_model(tr_f1.detach(), tr_f2.detach(), val_f1.detach(), val_f2.detach(),
                                                  clf, patience, 300, train_y, valid_y, inter_ind, optimizer_f)
            if f_val_loss < best_val:
                best_val = f_val_loss
                res1, res2 = best(te_f1.detach(), te_f2.detach())
                y_pred1 = torch.sigmoid(res1).detach().cpu()
                y_pred2 = torch.sigmoid(res2).detach().cpu()

                # Task1 performance
                fpr, tpr, threshold = metrics.roc_curve(test_y, y_pred1)
                roc_auc1 = metrics.auc(fpr, tpr)

                # Task2 performance
                fpr, tpr, threshold = metrics.roc_curve(test_y, y_pred2)
                roc_auc2 = metrics.auc(fpr, tpr)

        t1_auc.append(roc_auc1)
        print("Task1 test AUC:", roc_auc1)
        t2_auc.append(roc_auc2)
        print("Task2 test AUC:", roc_auc2)

    t1_res.append(sum(t1_auc) / len(t1_auc))
    t2_res.append(sum(t2_auc) / len(t2_auc))

    # Average performance at each epoch
    print("Task1: AUC", sum(t1_auc) / len(t1_auc))
    print("Task2: AUC", sum(t2_auc) / len(t2_auc))

# Final performance
print("-----------------------------------")
print("Task1 Average: AUC", sum(t1_res) / len(t1_res), np.std(t1_res))
print("Task2 Average: AUC", sum(t2_res) / len(t2_res), np.std(t2_res))

------------- Fold : 1-------------


100%|██████████| 3/3 [31:14<00:00, 624.88s/it]


Task1 test AUC: 0.8548710242925119
Task2 test AUC: 0.852335336839469
------------- Fold : 2-------------


100%|██████████| 3/3 [27:12<00:00, 544.10s/it]


Task1 test AUC: 0.8195592286501376
Task2 test AUC: 0.8280428249436513
------------- Fold : 3-------------


...

Task1 test AUC: 0.8919515401953418
Task2 test AUC: 0.8901984723265715
------------- Fold : 4-------------


  0%|          | 0/3 [00:00<?, ?it/s]

Task1 test AUC: 0.8312359128474831
Task2 test AUC: 0.8330046331079389
------------- Fold : 5-------------


100%|██████████| 3/3 [31:10<00:00, 623.47s/it]

Task1 test AUC: 0.8347577009767093
Task2 test AUC: 0.8343820435762584
Task1: AUC 0.849301903330829
Task2: AUC 0.8493269471575257
-----------------------------------
Task1 Average: AUC 0.8455853994490358 0.0023354978898925335
Task2 Average: AUC 0.8461241547708489 0.0028993565091317815
